In [1]:
import os

import pandas as pd
import numpy as np

from src.webscraping import (
    activate_web_driver,
    get_todays_matchups,
)

In [3]:
#retrieve list of teams playing today

driver = activate_web_driver('firefox')

# get today's games on NBA schedule
teams_list = get_todays_matchups(driver)

team_count = len(teams_list) 
matchups = []
for i in range(0,team_count,2):
    matchups.append([teams_list[i],teams_list[i+1]])

matchups




[WDM] - Downloading: 19.0kB [00:00, 6.52MB/s]                   


['/team/1610612743/nuggets/', '/team/1610612740/pelicans/', '/team/1610612756/suns/', '/team/1610612759/spurs/', '/team/1610612738/celtics/', '/team/1610612751/nets/', '/team/1610612763/grizzlies/', '/team/1610612765/pistons/', '/team/1610612739/cavaliers/', '/team/1610612752/knicks/', '/team/1610612747/lakers/', '/team/1610612764/wizards/', '/team/1610612741/bulls/', '/team/1610612758/kings/', '/team/1610612754/pacers/', '/team/1610612757/blazers/']


[['/team/1610612743/nuggets/', '/team/1610612740/pelicans/'],
 ['/team/1610612756/suns/', '/team/1610612759/spurs/'],
 ['/team/1610612738/celtics/', '/team/1610612751/nets/'],
 ['/team/1610612763/grizzlies/', '/team/1610612765/pistons/'],
 ['/team/1610612739/cavaliers/', '/team/1610612752/knicks/'],
 ['/team/1610612747/lakers/', '/team/1610612764/wizards/'],
 ['/team/1610612741/bulls/', '/team/1610612758/kings/'],
 ['/team/1610612754/pacers/', '/team/1610612757/blazers/']]